<font size=6> __Connecting to MS SQL Server using sqlalchemy__</font>

<font size=3>Notice: This script will not execute in google colab</font>

<font size=3>Notice: This script is for connecting via Windows Authentication</font>

# Imports

In [43]:
import pandas as pd
from sqlalchemy import create_engine, text, update

# Establishing connection

## Set server

In [85]:
server = '********\SQLEXPRESS'
# To find your server name, go to SSMS and execute: SELECT @@servername; it should look something like: 'NAME\SQLEXPRESS' 

## Set database

In [86]:
database = 'tempdb'
# Naturally you can also choose any other database that exists in your sql server

## Set driver

In [87]:
driver = 'ODBC Driver 17 for SQL Server'

In [88]:
# If you're not sure which driver to use, you can see a list of drivers by executing:
import pyodbc
pyodbc.drivers()

# There's a good chance your driver is either:
# 'ODBC Driver 17 for SQL Server'
# or
# 'SQL Server Native Client 11.0'

['SQL Server',
 'MySQL ODBC 8.0 ANSI Driver',
 'MySQL ODBC 8.0 Unicode Driver',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

## Set connection string

In [89]:
conx_string = f'mssql+pyodbc://{server}/{database}?driver={driver}'
print(conx_string)

mssql+pyodbc://********\SQLEXPRESS/tempdb?driver=ODBC Driver 17 for SQL Server


In [49]:
engine = create_engine(conx_string)

In [50]:
# create a SQLAlchemy connection object
conn = engine.connect()

# Executing SQL queries

## Define a function for executing SQL query

In [51]:
def execute_SQL(query_SQL):
    with engine.connect() as conn:
        stmt = text(query_SQL)
        conn.execute(stmt)
        conn.commit()

## Create

In [52]:
create_tryout1_SQL = '''
CREATE TABLE tryout1 (try_id INT, try_name VARCHAR(50))
'''

In [53]:
execute_SQL(create_tryout1_SQL)

In [54]:
# Check your tempdb database to find tryout1 was created (although still empty).

## Insert

In [55]:
insert_tryout1_SQL = '''
INSERT INTO tryout1
VALUES
  (1, 'This was a triumph')
, (2, 'I''m making a note here: Huge success!')
, (3, 'It''s hard to overstate my satisfaction.')
'''

In [56]:
execute_SQL(insert_tryout1_SQL)

In [57]:
# Check your tempdb database to find tryout1 now has data.

## Update

In [58]:
update_tryout1_SQL = '''
UPDATE tryout1
SET try_id = 100
WHERE try_id = 3
'''

In [59]:
execute_SQL(update_tryout1_SQL)

In [60]:
# Check your tempdb database to find try_id 3 is now 100.

## Read

In [61]:
# For reading we would need to do something a little different

In [62]:
select_tryout1_SQL = '''
SELECT *
FROM tryout1
WHERE try_id >= 2
'''

In [63]:
# If we just do that:
tryout1_df = execute_SQL(select_tryout1_SQL)

In [64]:
# We get nothing (because our `execute_SQL` function has no return...)
print(tryout1_df)
print(type(tryout1_df))

None
<class 'NoneType'>


In [65]:
# Instead, we use the `read_sql_query` function.
# Remember - our query has to pass as text()
tryout1_df = pd.read_sql_query(text(select_tryout1_SQL), engine.connect())

In [66]:
tryout1_df

,try_id,try_name
0,2,I'm making a note here: Huge success!
1,100,It's hard to overstate my satisfaction.


In [67]:
# Let's wrap it in a form of a function
def read_SQL(query_SQL):
    res_df = pd.read_sql_query(text(query_SQL), engine.connect())
    return res_df 

In [68]:
tryout1_df_by_func = read_SQL(select_tryout1_SQL)

In [69]:
tryout1_df_by_func

,try_id,try_name
0,2,I'm making a note here: Huge success!
1,100,It's hard to overstate my satisfaction.


## Drop

In [70]:
drop_tryout1_SQL = '''
DROP TABLE tryout1
'''

In [71]:
execute_SQL(drop_tryout1_SQL)

In [72]:
# Check your tempdb database to find tryout1 does not exist.

## Loading a DataFrame into MS SQL Server

In [73]:
# This is actually extremely easy, we just use the to_sql() method

In [74]:
# Getting some data
ny_baby = pd.read_csv(r'https://data.cityofnewyork.us/api/views/25th-nujf/rows.csv?accessType=DOWNLOAD')

In [75]:
ny_baby.sample(5)

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
15200,2014,MALE,WHITE NON HISPANIC,Leonardo,36,76
43388,2012,MALE,WHITE NON HISP,ROMAN,16,88
22904,2014,FEMALE,WHITE NON HISPANIC,Parker,17,79
5127,2012,MALE,ASIAN AND PACI,ROHAN,20,51
19326,2012,MALE,HISPANIC,HUNTER,12,98


In [76]:
# Loading it into SQL
ny_baby.to_sql('ny_baby', con=engine, if_exists='replace', index=False)

300

In [77]:
# Check your tempdb database to find ny_baby table with all its content.

# Extras

## Improved functions (try / except)

In [90]:
def execute_SQL(query_SQL):
    try:
        with engine.connect() as conn:
            stmt = text(query_SQL)
            conn.execute(stmt)
            conn.commit()
            conn.close()
            print('Execution done')
    except Exception as e:
        print(e)

In [91]:
def read_SQL(query_SQL):
    try:
        res_df = pd.read_sql_query(text(query_SQL), engine.connect())
        conn.close()
        print('Query reading done')
        return res_df
    except Exception as e:
        print(e)